# calculating the RM for each stress-testing scenario

This notebook calucates how different policy is performing in terms of planning reserve margin under different stress testing scenarios 

this notebook take the state of charge for battery resources into account but not the curtailment of solar and wind

In [7]:
import os
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path
import logging



In [8]:
# Load the CSV file into a DataFrame
linkages = pd.read_csv('/Users/liyangwang/kitxDMDU_analysis/2045-Core_test/filtered_linkages.csv')

# Filter the DataFrame
filteredLinkages = linkages[(linkages['linkage'] == 'ResourceToZone') & (linkages['component_to'] == 'CAISO')]

filteredLinkages

,component_from,linkage,scenario,component_to
41,CAISO_Hydro,ResourceToZone,Base,CAISO
6847,CAISO_Shed_DR_Existing,ResourceToZone,Shed DR LSE Plans: No LSE Plans,CAISO
6848,CAISO_Shed_DR_Existing,ResourceToZone,Shed DR LSE Plans: 25 MMT,CAISO
6849,CAISO_Shed_DR_Existing,ResourceToZone,Shed DR LSE Plans: 30 MMT,CAISO
6850,Shed_ag_Pumping,ResourceToZone,Base,CAISO
...,...,...,...,...
8497,Flex_LDV_V2G_Com,ResourceToZone,Mid_LDV,CAISO
8498,Flex_LDV_V1G_Res,ResourceToZone,High_LDV,CAISO
8499,Flex_LDV_V1G_Com,ResourceToZone,High_LDV,CAISO
8500,Flex_LDV_V2G_Res,ResourceToZone,High_LDV,CAISO


In [3]:
import logging

def setup_logging(log_file_path):
    logging.basicConfig(filename=log_file_path, level=logging.INFO, 
                        format='%(asctime)s - %(levelname)s - %(message)s')

def process_rm(policy_folder, output_file_path, log_file_path):
    setup_logging(log_file_path)
    reserveMarginList = []

    for root, dirs, files in os.walk(policy_folder):
        if 'expressions' in dirs:
            try:
                expressions_dir = os.path.join(root, 'expressions')
                parameters_dir = os.path.join(root, 'parameters')
                rep_periods_dir = os.path.join(root, 'temporal_settings')
                variables_dir = os.path.join(root, 'variables')
                temporal_settings_dir = os.path.join(root, 'temporal_settings')
                
                plant_capacity_file = os.path.join(expressions_dir, 'Plant_Provide_Power_Capacity_In_Timepoint_MW.csv')
                input_load_file = os.path.join(parameters_dir, 'input_load_mw.csv')
                soc_intra_period_file = os.path.join(variables_dir, 'SOC_Intra_Period.csv')
                provide_power_mw_file = os.path.join(variables_dir, 'Provide_Power_MW.csv')
                rep_periods_file = os.path.join(temporal_settings_dir, 'rep_periods.csv')
                transmit_power_mw_file = os.path.join(variables_dir, 'Transmit_Power_MW.csv')

                plantCapacity = pd.read_csv(plant_capacity_file)
                inputLoad = pd.read_csv(input_load_file)
                socIntra = pd.read_csv(soc_intra_period_file)
                providePower = pd.read_csv(provide_power_mw_file)
                repPeriod = pd.read_csv(rep_periods_file)
                transmitPower = pd.read_csv(transmit_power_mw_file)

                logging.info(f"Loaded CSV files from {root}")

                filteredPlantCapacity = plantCapacity[plantCapacity['PLANTS_THAT_PROVIDE_POWER'].isin(filteredLinkages['component_from'])]
                resources_with_storage = socIntra['RESOURCES_WITH_STORAGE'].unique()

                truePlantCapacity = pd.merge(
                    filteredPlantCapacity,
                    providePower,
                    on=['MODEL_YEARS', 'REP_PERIODS', 'HOURS', 'PLANTS_THAT_PROVIDE_POWER'],
                    how='inner'
                )
                mask = truePlantCapacity['PLANTS_THAT_PROVIDE_POWER'].isin(resources_with_storage)
                # finding the storage resources and incorporate state of charge into account, 
                # but this doesn't take curtailment into account for solar and wind resources
                truePlantCapacity.loc[mask, 'Plant_Provide_Power_Capacity_In_Timepoint_MW'] = truePlantCapacity.loc[mask, 'Provide_Power_MW']

                try:
                    truePlantCapacity = pd.merge(truePlantCapacity, repPeriod[['period', '0']], left_on='REP_PERIODS', right_on='period', how='left')
                    truePlantCapacity['datetime'] = pd.to_datetime(truePlantCapacity['MODEL_YEARS'].astype(str) + '-' + truePlantCapacity['0'].str[5:], errors='coerce')
                    truePlantCapacity['datetime'] = truePlantCapacity['datetime'] + pd.to_timedelta(truePlantCapacity['HOURS'], unit='h')
                    truePlantCapacity.drop(columns=['period', '0'], inplace=True)
                    logging.info("Converted MODEL_YEARS, REP_PERIODS, and HOURS into datetime successfully.")
                except Exception as e:
                    logging.error(f"Error converting MODEL_YEARS, REP_PERIODS, and HOURS into datetime: {e}")

                try:
                    caisoLoad = inputLoad[inputLoad['ZONES'] == 'CAISO'].copy()
                    caisoLoad[['REP_PERIODS', 'HOURS', 'MODEL_YEARS']] = caisoLoad[['REP_PERIODS', 'HOURS', 'MODEL_YEARS']].astype(int)
                    caisoLoad = pd.merge(caisoLoad, repPeriod[['period', '0']], left_on='REP_PERIODS', right_on='period', how='left')
                    caisoLoad['datetime'] = pd.to_datetime(caisoLoad['MODEL_YEARS'].astype(str) + '-' + caisoLoad['0'].str[5:], errors='coerce') + pd.to_timedelta(caisoLoad['HOURS'], unit='h')
                    caisoLoad = caisoLoad.sort_values(by='datetime')
                    logging.info("Calculated hourly input load successfully.")
                except Exception as e:
                    logging.error(f"Error calculating hourly input load: {e}")
                    
                try:
                    caisoImportExport = transmitPower[transmitPower['TRANSMISSION_LINES'].str.contains('to CAISO')].copy()
                    caisoImportExport[['REP_PERIODS', 'HOURS', 'MODEL_YEARS']] = caisoImportExport[['REP_PERIODS', 'HOURS', 'MODEL_YEARS']].astype(int)
                    caisoImportExport = pd.merge(caisoImportExport, repPeriod[['period', '0']], left_on='REP_PERIODS', right_on='period', how='left')
                    caisoImportExport['datetime'] = pd.to_datetime(caisoImportExport['MODEL_YEARS'].astype(str) + '-' + caisoImportExport['0'].str[5:], errors='coerce') + pd.to_timedelta(caisoImportExport['HOURS'], unit='h')
                    caisoImportExport = caisoImportExport.sort_values(by='datetime')
                    caisoImportExport = caisoImportExport.groupby('datetime')['Transmit_Power_MW'].sum().reset_index()
                    logging.info("Calculated hourly import/export successfully.")
                except Exception as e:
                    logging.error(f"Error calculating import/export : {e}")

                try:
                    reserveMargin = truePlantCapacity.groupby('datetime')['Plant_Provide_Power_Capacity_In_Timepoint_MW'].sum().reset_index()
                    reserveMargin = reserveMargin.merge(caisoImportExport, on='datetime')
                    reserveMargin = reserveMargin.merge(caisoLoad, on='datetime')
                    reserveMargin['PRM'] = ((reserveMargin['Plant_Provide_Power_Capacity_In_Timepoint_MW'] + reserveMargin['Transmit_Power_MW']  - reserveMargin['input_load_mw']) / reserveMargin['input_load_mw']) * 100
                    logging.info("Calculated reserve margin successfully.")
                except Exception as e:
                    logging.error(f"Error calculating reserve margin: {e}")

                parts = root.split('/')
                part1 = parts[-3].replace('2010-2020ML', '-')
                part2 = parts[-2].replace('2045-', '')
                case_name = f"{part1}{part2}"
                reserveMargin['case'] = case_name
                reserveMarginList.append(reserveMargin)
            except FileNotFoundError as e:
                logging.error(f"File not found: {e}")
                continue

    RMall_df = pd.concat(reserveMarginList, ignore_index=True)
    RMall_df.drop(columns=['ZONES', 'MODEL_YEARS', 'REP_PERIODS', 'HOURS', 'period', '0'], inplace=True)
    RMall_df.to_csv(output_file_path, index=False)
    logging.info(f"Saved RMall_df to {log_file_path}")
    return RMall_df

In [4]:
Core = '/Users/liyangwang/kitxDMDU_analysis/2045-Core_test'  # Replace with your root directory
CoreLog = '/Users/liyangwang/kitxDMDU_analysis/coretest.log'
CorePRM = process_rm(Core, '/Users/liyangwang/kitxDMDU_analysis/2045-Core_test_PRM.csv',CoreLog)


CorePRM

,datetime,Plant_Provide_Power_Capacity_In_Timepoint_MW,Transmit_Power_MW,input_load_mw,PRM,case
0,2045-01-01 00:00:00,47953.201,905.843,46159.888,5.847406,Baseline2_45_CESM2_2016
1,2045-01-01 01:00:00,47488.039,635.384,44710.867,7.632498,Baseline2_45_CESM2_2016
2,2045-01-01 02:00:00,41715.592,1328.683,39829.731,8.070715,Baseline2_45_CESM2_2016
3,2045-01-01 03:00:00,37266.118,2301.159,35554.393,11.286605,Baseline2_45_CESM2_2016
4,2045-01-01 04:00:00,35455.182,1756.286,33076.003,12.502916,Baseline2_45_CESM2_2016
...,...,...,...,...,...,...
26275,2045-12-31 19:00:00,37705.291,3592.131,37700.782,9.539961,Baseline2_45_CESM2_2017
26276,2045-12-31 20:00:00,38039.663,3254.343,37753.060,9.379229,Baseline2_45_CESM2_2017
26277,2045-12-31 21:00:00,38705.301,3622.192,39128.303,8.176153,Baseline2_45_CESM2_2017
26278,2045-12-31 22:00:00,36950.061,4537.209,39109.491,6.079800,Baseline2_45_CESM2_2017


In [5]:
Core = '/global/scratch/users/liyangwang/Robustness/reports/resolve/2045-Core_25MMT'  # Replace with your root directory
highGas = '/global/scratch/users/liyangwang/Robustness/reports/resolve/2045-High_Gas_Ret'  # Replace with your root directory
leastCost = '/global/scratch/users/liyangwang/Robustness/reports/resolve/2045-LeastCost_25MMT_Sens'  # Replace with your root directory

In [6]:
CoreLog = '/global/scratch/users/liyangwang/Robustness/z.output/PRMv2/core.log'
HGLog = '/global/scratch/users/liyangwang/Robustness/z.output/PRMv2/highGas.log'
LCLog = '/global/scratch/users/liyangwang/Robustness/z.output/PRMv2/leastCost.log'

In [7]:
CorePRM = process_rm(Core, '/global/scratch/users/liyangwang/Robustness/z.output/PRMv2/2045-Core_25MMT_PRM.csv',CoreLog)
highGasPRM = process_rm(highGas, '/global/scratch/users/liyangwang/Robustness/z.output/PRMv2/2045-High_Gas_Ret_PRM.csv', HGLog)
leastCostPRM = process_rm(leastCost, '/global/scratch/users/liyangwang/Robustness/z.output/PRMv2/2045-LeastCost_25MMT_Sens_PRM.csv',LCLog)


In [50]:
vulnerabeHours = CorePRM[CorePRM['PRM'] < 0].groupby('case').size().reset_index(name='hours_below_17_prm')

vulnerabeHours

                            case  hours_below_17_prm
0        Baseline2_45_CESM2_2015                1102
1        Baseline2_45_CESM2_2016                 821
2        Baseline2_45_CESM2_2017                 903
3  highDemandslim2_45_CESM2_2015                5620


In [8]:
transmitPower

,TRANSMISSION_LINES,MODEL_YEARS,REP_PERIODS,HOURS,Transmit_Power_MW
0,BANC to CAISO,2045,0,0,1231.761
1,BANC to CAISO,2045,0,1,1918.330
2,BANC to CAISO,2045,0,2,2041.173
3,BANC to CAISO,2045,0,3,1437.308
4,BANC to CAISO,2045,0,4,1370.046
...,...,...,...,...,...
122635,SW to LDWP,2045,364,19,0.000
122636,SW to LDWP,2045,364,20,0.000
122637,SW to LDWP,2045,364,21,0.000
122638,SW to LDWP,2045,364,22,0.000


# debug zone

In [2]:
root ='/global/scratch/users/liyangwang/Robustness/reports/resolve/2045-Core_25MMT/Baseline2010-2020ML/2045-2_45_CESM2_2015/2024-08-16 10-48-13'

In [9]:
expressions_dir = os.path.join(root, 'expressions')
parameters_dir = os.path.join(root, 'parameters')
rep_periods_dir = os.path.join(root, 'temporal_settings')
variables_dir = os.path.join(root, 'variables')
temporal_settings_dir = os.path.join(root, 'temporal_settings')

plant_capacity_file = os.path.join(expressions_dir, 'Plant_Provide_Power_Capacity_In_Timepoint_MW.csv')
input_load_file = os.path.join(parameters_dir, 'input_load_mw.csv')
soc_intra_period_file = os.path.join(variables_dir, 'SOC_Intra_Period.csv')
provide_power_mw_file = os.path.join(variables_dir, 'Provide_Power_MW.csv')
rep_periods_file = os.path.join(temporal_settings_dir, 'rep_periods.csv')
transmit_power_mw_file = os.path.join(variables_dir, 'Transmit_Power_MW.csv')

plantCapacity = pd.read_csv(plant_capacity_file)
inputLoad = pd.read_csv(input_load_file)
socIntra = pd.read_csv(soc_intra_period_file)
providePower = pd.read_csv(provide_power_mw_file)
repPeriod = pd.read_csv(rep_periods_file)
transmitPower = pd.read_csv(transmit_power_mw_file)

logging.info(f"Loaded CSV files from {root}")

filteredPlantCapacity = plantCapacity[plantCapacity['PLANTS_THAT_PROVIDE_POWER'].isin(filteredLinkages['component_from'])]
resources_with_storage = socIntra['RESOURCES_WITH_STORAGE'].unique()

truePlantCapacity = pd.merge(
    filteredPlantCapacity,
    providePower,
    on=['MODEL_YEARS', 'REP_PERIODS', 'HOURS', 'PLANTS_THAT_PROVIDE_POWER'],
    how='inner'
)
mask = truePlantCapacity['PLANTS_THAT_PROVIDE_POWER'].isin(resources_with_storage)
# finding the storage resources and incorporate state of charge into account, 
# but this doesn't take curtailment into account for solar and wind resources
truePlantCapacity.loc[mask, 'Plant_Provide_Power_Capacity_In_Timepoint_MW'] = truePlantCapacity.loc[mask, 'Provide_Power_MW']

try:
    truePlantCapacity = pd.merge(truePlantCapacity, repPeriod[['period', '0']], left_on='REP_PERIODS', right_on='period', how='left')
    truePlantCapacity['datetime'] = pd.to_datetime(truePlantCapacity['MODEL_YEARS'].astype(str) + '-' + truePlantCapacity['0'].str[5:], errors='coerce')
    truePlantCapacity['datetime'] = truePlantCapacity['datetime'] + pd.to_timedelta(truePlantCapacity['HOURS'], unit='h')
    truePlantCapacity.drop(columns=['period', '0'], inplace=True)
    logging.info("Converted MODEL_YEARS, REP_PERIODS, and HOURS into datetime successfully.")
except Exception as e:
    logging.error(f"Error converting MODEL_YEARS, REP_PERIODS, and HOURS into datetime: {e}")

try:
    caisoLoad = inputLoad[inputLoad['ZONES'] == 'CAISO'].copy()
    caisoLoad[['REP_PERIODS', 'HOURS', 'MODEL_YEARS']] = caisoLoad[['REP_PERIODS', 'HOURS', 'MODEL_YEARS']].astype(int)
    caisoLoad = pd.merge(caisoLoad, repPeriod[['period', '0']], left_on='REP_PERIODS', right_on='period', how='left')
    caisoLoad['datetime'] = pd.to_datetime(caisoLoad['MODEL_YEARS'].astype(str) + '-' + caisoLoad['0'].str[5:], errors='coerce') + pd.to_timedelta(caisoLoad['HOURS'], unit='h')
    caisoLoad = caisoLoad.sort_values(by='datetime')
    logging.info("Calculated hourly input load successfully.")
except Exception as e:
    logging.error(f"Error calculating hourly input load: {e}")
    
try:
    caisoImportExport = transmitPower[transmitPower['TRANSMISSION_LINES'].str.contains('to CAISO')].copy()
    caisoImportExport[['REP_PERIODS', 'HOURS', 'MODEL_YEARS']] = caisoImportExport[['REP_PERIODS', 'HOURS', 'MODEL_YEARS']].astype(int)
    caisoImportExport = pd.merge(caisoImportExport, repPeriod[['period', '0']], left_on='REP_PERIODS', right_on='period', how='left')
    caisoImportExport['datetime'] = pd.to_datetime(caisoImportExport['MODEL_YEARS'].astype(str) + '-' + caisoImportExport['0'].str[5:], errors='coerce') + pd.to_timedelta(caisoImportExport['HOURS'], unit='h')
    caisoImportExport = caisoImportExport.sort_values(by='datetime')
    caisoImportExport = caisoImportExport.groupby('datetime')['Transmit_Power_MW'].sum().reset_index()
    logging.info("Calculated hourly import/export successfully.")
except Exception as e:
    logging.error(f"Error calculating import/export : {e}")

try:
    reserveMargin = truePlantCapacity.groupby('datetime')['Plant_Provide_Power_Capacity_In_Timepoint_MW'].sum().reset_index()
    reserveMargin = reserveMargin.merge(caisoImportExport, on='datetime')
    reserveMargin = reserveMargin.merge(caisoLoad, on='datetime')
    reserveMargin['PRM'] = ((reserveMargin['Plant_Provide_Power_Capacity_In_Timepoint_MW'] + reserveMargin['Transmit_Power_MW']  - reserveMargin['input_load_mw']) / reserveMargin['input_load_mw']) * 100
    logging.info("Calculated reserve margin successfully.")
except Exception as e:
    logging.error(f"Error calculating reserve margin: {e}")

parts = root.split('/')
part1 = parts[-3].replace('2010-2020ML', '-')
part2 = parts[-2].replace('2045-', '')
case_name = f"{part1}{part2}"
reserveMargin['case'] = case_name

In [10]:
reserveMargin

,datetime,Plant_Provide_Power_Capacity_In_Timepoint_MW,Transmit_Power_MW,ZONES,MODEL_YEARS,REP_PERIODS,HOURS,input_load_mw,period,0,PRM,case
0,2045-01-01 00:00:00,43355.406,5613.877,CAISO,2045,0,0,45428.334,0,2015-01-01,7.794583,Baseline-2_45_CESM2_2015
1,2045-01-01 01:00:00,40984.366,6415.875,CAISO,2045,0,1,44053.125,0,2015-01-01,7.597908,Baseline-2_45_CESM2_2015
2,2045-01-01 02:00:00,36493.329,6298.566,CAISO,2045,0,2,39103.021,0,2015-01-01,9.433731,Baseline-2_45_CESM2_2015
3,2045-01-01 03:00:00,34172.467,5328.104,CAISO,2045,0,3,34990.282,0,2015-01-01,12.890119,Baseline-2_45_CESM2_2015
4,2045-01-01 04:00:00,32244.839,4984.941,CAISO,2045,0,4,32568.934,0,2015-01-01,14.310711,Baseline-2_45_CESM2_2015
...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2045-12-31 19:00:00,42485.859,2981.715,CAISO,2045,364,19,41653.489,364,2015-12-31,9.156700,Baseline-2_45_CESM2_2015
8756,2045-12-31 20:00:00,43093.912,3068.426,CAISO,2045,364,20,42757.628,364,2015-12-31,7.962813,Baseline-2_45_CESM2_2015
8757,2045-12-31 21:00:00,44055.974,2990.855,CAISO,2045,364,21,43764.700,364,2015-12-31,7.499489,Baseline-2_45_CESM2_2015
8758,2045-12-31 22:00:00,43390.743,2716.833,CAISO,2045,364,22,43607.221,364,2015-12-31,5.733810,Baseline-2_45_CESM2_2015


In [11]:
truePlantCapacity

,PLANTS_THAT_PROVIDE_POWER,MODEL_YEARS,REP_PERIODS,HOURS,Plant_Provide_Power_Capacity_In_Timepoint_MW,Provide_Power_MW,datetime
0,Arizona_Solar,2045,0,0,0.000,0.000,2045-01-01 00:00:00
1,Arizona_Solar,2045,0,1,0.000,0.000,2045-01-01 01:00:00
2,Arizona_Solar,2045,0,2,0.000,0.000,2045-01-01 02:00:00
3,Arizona_Solar,2045,0,3,0.000,0.000,2045-01-01 03:00:00
4,Arizona_Solar,2045,0,4,0.000,0.000,2045-01-01 04:00:00
...,...,...,...,...,...,...,...
805915,Wyoming_Wind,2045,364,19,4334.708,4334.708,2045-12-31 19:00:00
805916,Wyoming_Wind,2045,364,20,5008.996,5008.996,2045-12-31 20:00:00
805917,Wyoming_Wind,2045,364,21,5586.957,5586.957,2045-12-31 21:00:00
805918,Wyoming_Wind,2045,364,22,6164.918,6164.918,2045-12-31 22:00:00
